Web Crawler and Scraper for TAG internal WIKI.  Login uses Selenium and BeautifulSoup is used for the actual scraping.   Consists of a Web Crawler and Scraper for locating and scraping all Wiki links.     Data is locally stored in iFrames with Page description being stored in reg HTML format, so last Paragraph is stored as the page category and the iFrame as the data.   

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import html
import requests
import re


#path to DB
output_file_path=""


# URL's 
login_url =   '        
base_url =  ''
start_url = base_url + '/wiki'      

# Set of visited URLs
visited = set()


# login credentials
username = "redacted"
password = "redacted"

# browser instance
driver = webdriver.Chrome()



# Open the login page
driver.get(login_url)

# Find the username and password input fields and fill them in
username_field = driver.find_element( By.CSS_SELECTOR, 'input[type="text"]')
password_field = driver.find_element(By.CSS_SELECTOR, 'input[type="password"]')

username_field.send_keys(username)
password_field.send_keys(password)

# Submit the login form
password_field.send_keys(Keys.RETURN)

# Wait for the login to complete
wait = WebDriverWait(driver, 10)
wait.until(EC.url_changes(login_url))




# Transfer the session cookies from the Selenium driver to the requests session
cookies = driver.get_cookies()
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])


exceptions=["iOS","iPhone","BlackBerry","NetNanny","mSpy"]        
#Add spaces between sentences.  Required for Data Cleaning    
def add_spaces(text, exclusions):
    
    # Use regular expression to insert spaces before words that start with a capital letter
    pattern = r'(?<=[a-z])(?=[A-Z])(?!(?:' + '|'.join(re.escape(ex) for ex in exceptions) + '))'
    modified_text = re.sub(pattern, ' ', text)
    return modified_text    
    

    

#Function for crawling and scraping all Wiki Sites
def scrape_page(url):
    
   
    
    response = session.get(url)
    if response.status_code == 200:   
        visited.add(url)
        
        # Parse the page using BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the iframe and Title element
        iframe_data = soup.find_all("iframe")
        header=soup.find_all('p')[-1]
        
        
        with open(output_file_path, "a", encoding="utf-8") as output_file:
            
        #Store the page Title (looks for last 'p' element)
             for headers in header:
              
                output_file.write('\n\n** Data from the '+ headers.get_text()+' page **\n')
                
        # Loop through the locally stored iframe elements
             for iframe in iframe_data:
                srcdoc_content = iframe["srcdoc"]
                decoded_content = html.unescape(srcdoc_content)
                iframe_soup = BeautifulSoup(decoded_content, "html.parser")
            
            
        # Store the content of the iframe
              
                output_file.write(add_spaces(iframe_soup.get_text(),exceptions))
                output_file.write("-" * 20)
              

    # Find links on the page and follow them
        for link in soup.find_all('a', href=True):
            next_url = link['href']
            
            #Check that the link is local and part of the wiki
            if 'wiki' in link.text.lower() or 'wiki' in next_url.lower():
                if next_url.startswith('/'):  # Relative URLs
                        next_url = base_url + next_url
                        
                        #If not yet scraped, scrape
                if next_url.startswith(base_url) and next_url not in visited:
                        scrape_page(next_url)




# Start scraping from the main URL
if __name__ == "__main__":

    scrape_page(start_url)
    
 # Once finished,end Selenium session 
driver.quit()

print('Done!')
    
    


      


        
        



Done!
